<a href="https://colab.research.google.com/github/Maryann-01/ML-Projects/blob/main/Zindi_Urban_Air_Pollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

Create Wrangle function

In [3]:
def wrangle(filepath):
  df=pd.read_csv(filepath)
  # Dropping columns with more than 50% missing values
  missing_val_per=df.isna().sum()/len(df)
  columns_to_drop=missing_val_per[missing_val_per>=0.5].index
  df.drop(columns=columns_to_drop,inplace=True)

  # Dropping columns that leak info about the target value
  df.drop(columns=["target_min","target_max","target_variance","target_count"],inplace=True)

  return df

In [4]:
train_data=wrangle("/content/Train.csv")
# train_data.head()
# train_data.info()
# test_data=wrangle("/content/Test.csv")
# test_data.info()

Dropping columns with >50% missing values

In [5]:
# train_data.isna().sum().sort_values(ascending=False)
# len(train_data)
missing_val_per=train_data.isna().sum()/len(train_data)
missing_val_per.tail(30)
columns_to_drop=missing_val_per[missing_val_per>=0.5]
columns_to_drop

,0


In [6]:
X_train= train_data.drop('target', axis=1)
y_train = train_data['target']

# X_train.head()

In [14]:
# Creating a customer transformer for the date column
class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['Date'] = pd.to_datetime(X['Date'])


        X['year'] = X['Date'].dt.year
        X['month'] = X['Date'].dt.month
        X['dayofweek'] = X['Date'].dt.dayofweek
        X['days_since_start'] = (X['Date'] - X['Date'].min()).dt.days

        # Optionally drop the original 'Date' column and drop the year column because it is the same year all through(low-cardinality)
        X = X.drop(columns=['Date', "year"])

        return X


In [15]:
# Initializing the transformer
date_feature_extractor = DateFeatureExtractor()
X_train_transformed = date_feature_extractor.transform(X_train)
X_train_transformed.head()


,Place_ID X Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,...,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,month,dayofweek,days_since_start
0,010Q650 X 2020-01-02,010Q650,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,...,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1,3,0
1,010Q650 X 2020-01-03,010Q650,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,...,0.000050,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1,4,1
2,010Q650 X 2020-01-04,010Q650,16.400000,33.400002,0.00750,27.031030,5.065727,3.500559,0.000067,0.000170,...,0.000053,-1.572950,0.082063,-105.201338,49.839714,-78.342701,34.296977,1,5,2
3,010Q650 X 2020-01-05,010Q650,6.911948,21.300001,0.00391,23.971857,3.004001,1.099468,0.000083,0.000175,...,0.000110,-1.239317,0.121261,-104.334066,29.181258,-73.896588,30.545446,1,6,3
4,010Q650 X 2020-01-06,010Q650,13.900001,44.700001,0.00535,16.816309,2.621787,2.670559,0.000070,0.000142,...,0.000159,0.202489,0.037919,58.850179,0.797294,-68.612480,26.899694,1,0,4


In [17]:
# Now, we search for high and low_cardinality columns

X_train_transformed.nunique().sort_values(ascending=False).tail(30)
# Drop Year because we have just one year


,0
L3_CO_CO_column_number_density,24802
L3_CO_H2O_column_number_density,24802
L3_CO_sensor_altitude,24793
L3_CO_cloud_height,24610
L3_SO2_solar_zenith_angle,22987
L3_SO2_solar_azimuth_angle,22987
L3_SO2_sensor_azimuth_angle,22987
L3_SO2_SO2_column_number_density_amf,22987
L3_SO2_SO2_slant_column_number_density,22987
L3_SO2_sensor_zenith_angle,22987
